In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [2]:
rr = RandomRules(
  TOTAL_NUMBERS = 40,
  CHOOSE = 6,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5, 6],
  MAX_B_VALUES = [35, 36, 37, 38, 39, 40]
)


In [3]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=42'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [13]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_b5 = None
post_str_blast = None
for row in rows:
  for col_index in [2, 3]:
    cols = [data.text.strip() for data in row.find_all('td')]
    if (cols[col_index] == ''):
      continue

    # cols = [data.text.strip() for data in row.find_all('td')]
    
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_b2)
    cols.append(post_str_b3)
    cols.append(post_str_b4)
    cols.append(post_str_b5)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_b2 = str_combo[1]
    post_str_b3 = str_combo[2]
    post_str_b4 = str_combo[3]
    post_str_b5 = str_combo[4]
    post_str_blast = str_combo[-1]



# lst = []
# for row in rows:
  
  
#   for col_index in [2, 3]:
#     cols = [data.text.strip() for data in row.find_all('td')]
#     if (cols[col_index] == ''):
#       continue
  
#     id, bolillas_int = formalize_data(bolillas_str=cols[col_index])
#     cols.append(id)
#     cols.append(bolillas_int)
#     cols.extend([b for b in bolillas_int])
    
#     is_rare, reason = rr.is_rare_combination(bolillas_int, id)
#     cols.append(is_rare)
#     cols.append(reason)
#     left_scores = [round(b, 4) for b in rr.calculate_left_scores(bolillas_int)]

#     right_scores = [round(b, 4) for b in rr.calculate_right_scores(bolillas_int)]
#     lr_scores = [round(left + right,4) for left, right in zip(left_scores, right_scores)]
#     row_score = round(sum(lr_scores),4)

#     shared_values = list(set(left_scores[0:5]) & set(right_scores[1:6]))
#     shared_values = [round(s, 4) for s in shared_values]
#     cols.extend([score for score in left_scores])
#     cols.extend([score for score in right_scores])
#     # cols.extend(lr_scores)
#     cols.append(shared_values)
#     cols.append(len(shared_values))
#     cols.append(row_score)
#     cols.append(rr.jumps_map(bolillas_int))
#     cols.append(rr.unijump(bolillas_int))
    
#     lst.append(cols)

with open('./kbl_cache_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas regulares', 'bolillas chauchamba', 'cc1', 'cc2',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=5280


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""",null,null,null,null,null,null,false,0,"""04_06_03_22_01""",-89
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""04_06_03_22_01""",-89
"""15/04/2025""","""1795""","""05 21 37 13 08 34""","""38 07 03 09 04 27""","""""","""""","""050813213437""","[5, 8, … 37]",5,8,13,21,34,37,"""05""","""08""","""13""","""21""","""34""","""37""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""03_05_08_13_03""",-18
"""15/04/2025""","""1795""","""05 21 37 13 08 34""","""38 07 03 09 04 27""","""""","""""","""050813213437""","[5, 8, … 37]",5,8,13,21,34,37,"""05""","""08""","""13""","""21""","""34""","""37""","""05""","""08""","""13""","""21""","""34""","""37""",false,0,"""03_05_08_13_03""",-18
"""12/04/2025""","""1794""","""10 32 26 03 19 11""","""39 07 28 21 05 12""","""""","""""","""031011192632""","[3, 10, … 32]",3,10,11,19,26,32,"""03""","""10""","""11""","""19""","""26""","""32""","""05""","""08""","""13""","""21""","""34""","""37""",false,0,"""07_01_08_07_06""",29
"""12/04/2025""","""1794""","""10 32 26 03 19 11""","""39 07 28 21 05 12""","""""","""""","""031011192632""","[3, 10, … 32]",3,10,11,19,26,32,"""03""","""10""","""11""","""19""","""26""","""32""","""03""","""10""","""11""","""19""","""26""","""32""",false,0,"""07_01_08_07_06""",29
"""10/04/2025""","""1793""","""01 36 20 18 22 02""","""33 17 29 39 30 36""","""""","""""","""010218202236""","[1, 2, … 36]",1,2,18,20,22,36,"""01""","""02""","""18""","""20""","""22""","""36""","""03""","""10""","""11""","""19""","""26""","""32""",true,8,"""01_16_02_02_14""",-45
"""10/04/2025""","""1793""","""01 36 20 18 22 02""","""33 17 29 39 30 36""","""""","""""","""010218202236""","[1, 2, … 36]",1,2,18,20,22,36,"""01""","""02""","""18""","""20""","""22""","""36""","""01""","""02""","""18""","""20""","""22""","""36""",true,8,"""01_16_02_02_14""",-45
"""08/04/2025""","""1792""","""17 39 38 04 40 28""","""23 34 17 08 24 01""","""""","""""","""041728383940""","[4, 17, … 40]",4,17,28,38,39,40,"""04""","""17""","""28""","""38""","""39""","""40""","""01""","""02""","""18""","""20""","""22""","""36""",true,8,"""13_11_10_01_01""",26


In [14]:
dfRows.item(0, 'b2')

6

In [15]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [16]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='02'
df_viz FOR str_b1 = 02


post_str_b1,count,percent
str,u32,f64
"""02""",278,41.063516
"""01""",73,10.782866
"""03""",69,10.192024
"""04""",44,6.499261
"""05""",31,4.579025
…,…,…
"""17""",4,0.590842
"""16""",3,0.443131
"""20""",2,0.295421


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 02
Last 3 results last_results=['04', '02']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""04_06_03_22_01""",-89
"""01/03/2025""","""1776""","""02 34 38 14 22 40""","""09 08 36 39 14 01""","""""","""""","""021422343840""","[2, 14, … 40]",2,14,22,34,38,40,"""02""","""14""","""22""","""34""","""38""","""40""","""04""","""06""","""11""","""18""","""26""","""30""",true,4,"""12_08_12_04_02""",38
"""01/03/2025""","""1776""","""02 34 38 14 22 40""","""09 08 36 39 14 01""","""""","""""","""021422343840""","[2, 14, … 40]",2,14,22,34,38,40,"""02""","""14""","""22""","""34""","""38""","""40""","""02""","""14""","""22""","""34""","""38""","""40""",true,4,"""12_08_12_04_02""",38
"""13/02/2025""","""1769""","""20 07 02 12 27 08""","""23 11 16 26 32 17""","""""","""""","""020708122027""","[2, 7, … 27]",2,7,8,12,20,27,"""02""","""07""","""08""","""12""","""20""","""27""","""08""","""16""","""24""","""29""","""33""","""39""",true,16,"""05_01_04_08_07""",0
"""13/02/2025""","""1769""","""20 07 02 12 27 08""","""23 11 16 26 32 17""","""""","""""","""020708122027""","[2, 7, … 27]",2,7,8,12,20,27,"""02""","""07""","""08""","""12""","""20""","""27""","""02""","""07""","""08""","""12""","""20""","""27""",true,16,"""05_01_04_08_07""",0
"""18/01/2025""","""1758""","""28 02 18 03 16 26""","""34 36 24 38 01 20""","""""","""""","""020316182628""","[2, 3, … 28]",2,3,16,18,26,28,"""02""","""03""","""16""","""18""","""26""","""28""","""06""","""08""","""16""","""19""","""20""","""38""",false,0,"""01_13_02_08_02""",-69
"""18/01/2025""","""1758""","""28 02 18 03 16 26""","""34 36 24 38 01 20""","""""","""""","""020316182628""","[2, 3, … 28]",2,3,16,18,26,28,"""02""","""03""","""16""","""18""","""26""","""28""","""02""","""03""","""16""","""18""","""26""","""28""",false,0,"""01_13_02_08_02""",-69
"""14/01/2025""","""1756""","""18 02 23 34 31 16""","""33 24 27 14 26 31""","""""","""""","""021618233134""","[2, 16, … 34]",2,16,18,23,31,34,"""02""","""16""","""18""","""23""","""31""","""34""","""07""","""10""","""18""","""19""","""37""","""39""",false,0,"""14_02_05_08_03""",7
"""14/01/2025""","""1756""","""18 02 23 34 31 16""","""33 24 27 14 26 31""","""""","""""","""021618233134""","[2, 16, … 34]",2,16,18,23,31,34,"""02""","""16""","""18""","""23""","""31""","""34""","""02""","""16""","""18""","""23""","""31""","""34""",false,0,"""14_02_05_08_03""",7


df_jumps_map FOR str_b1 = 02


jumps_map,count
str,u32
"""03_03_24_03_01""",2
"""02_06_02_03_05""",2
"""03_05_17_01_12""",2
"""05_03_09_09_04""",2
"""08_06_08_03_02""",2
…,…
"""07_18_01_07_01""",1
"""09_03_02_13_10""",1
"""04_10_06_08_03""",1


df_unijump FOR str_b1 = 02


unijump,count
i64,u32
3,14
0,12
-17,12
-37,11
-38,10
…,…
45,1
88,1
-65,1


In [17]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='06'
df_viz FOR str_b2 = 06


post_str_b2,count,percent
str,u32,f64
"""06""",106,36.933798
"""10""",15,5.226481
"""09""",14,4.878049
"""05""",14,4.878049
"""07""",13,4.529617
…,…,…
"""25""",1,0.348432
"""23""",1,0.348432
"""26""",1,0.348432


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 06
Last 3 results last_results=['06', '15']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""04_06_03_22_01""",-89
"""04/03/2025""","""1777""","""18 11 04 06 26 30""","""40 25 08 27 07 24""","""""","""""","""040611182630""","[4, 6, … 30]",4,6,11,18,26,30,"""04""","""06""","""11""","""18""","""26""","""30""","""05""","""15""","""18""","""29""","""30""","""39""",false,0,"""02_05_07_08_04""",-4
"""04/03/2025""","""1777""","""18 11 04 06 26 30""","""40 25 08 27 07 24""","""""","""""","""040611182630""","[4, 6, … 30]",4,6,11,18,26,30,"""04""","""06""","""11""","""18""","""26""","""30""","""04""","""06""","""11""","""18""","""26""","""30""",false,0,"""02_05_07_08_04""",-4
"""25/02/2025""","""1774""","""40 06 31 03 26 32""","""36 37 07 14 22 02""","""""","""""","""030626313240""","[3, 6, … 40]",3,6,26,31,32,40,"""03""","""06""","""26""","""31""","""32""","""40""","""03""","""07""","""08""","""22""","""25""","""29""",false,0,"""03_20_05_01_08""",-43
"""25/02/2025""","""1774""","""40 06 31 03 26 32""","""36 37 07 14 22 02""","""""","""""","""030626313240""","[3, 6, … 40]",3,6,26,31,32,40,"""03""","""06""","""26""","""31""","""32""","""40""","""03""","""06""","""26""","""31""","""32""","""40""",false,0,"""03_20_05_01_08""",-43
"""28/01/2025""","""1762""","""39 24 01 06 14 18""","""20 13 22 37 07 11""","""""","""""","""010614182439""","[1, 6, … 39]",1,6,14,18,24,39,"""01""","""06""","""14""","""18""","""24""","""39""","""04""","""20""","""31""","""33""","""36""","""38""",false,0,"""05_08_04_06_15""",-12
"""28/01/2025""","""1762""","""39 24 01 06 14 18""","""20 13 22 37 07 11""","""""","""""","""010614182439""","[1, 6, … 39]",1,6,14,18,24,39,"""01""","""06""","""14""","""18""","""24""","""39""","""01""","""06""","""14""","""18""","""24""","""39""",false,0,"""05_08_04_06_15""",-12
"""07/01/2025""","""1753""","""38 27 07 04 08 06""","""36 33 40 01 09 02""","""""","""""","""040607082738""","[4, 6, … 38]",4,6,7,8,27,38,"""04""","""06""","""07""","""08""","""27""","""38""","""02""","""11""","""14""","""18""","""22""","""39""",true,24,"""02_01_01_19_11""",-61
"""07/01/2025""","""1753""","""38 27 07 04 08 06""","""36 33 40 01 09 02""","""""","""""","""040607082738""","[4, 6, … 38]",4,6,7,8,27,38,"""04""","""06""","""07""","""08""","""27""","""38""","""04""","""06""","""07""","""08""","""27""","""38""",true,24,"""02_01_01_19_11""",-61


df_jumps_map FOR str_b2 = 06


jumps_map,count
str,u32
"""04_04_01_04_14""",2
"""02_13_04_02_03""",2
"""03_09_01_03_15""",2
"""04_14_10_05_03""",2
"""03_20_05_01_08""",2
…,…
"""05_07_11_02_14""",1
"""02_05_04_04_12""",1
"""05_02_21_01_08""",1


df_unijump FOR str_b2 = 06


unijump,count
i64,u32
-15,7
-1,6
19,6
-49,6
0,6
…,…
-37,1
-23,1
-22,1


In [18]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='12'
df_viz FOR str_b3 = 12


post_str_b3,count,percent
str,u32,f64
"""12""",75,35.377358
"""20""",13,6.132075
"""15""",11,5.188679
"""21""",11,5.188679
"""17""",11,5.188679
…,…,…
"""03""",1,0.471698
"""07""",1,0.471698
"""05""",1,0.471698


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 12
Last 3 results last_results=['19', '12']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""04_06_03_22_01""",-89
"""11/03/2025""","""1780""","""15 12 34 10 06 39""","""10 27 16 07 02 40""","""""","""""","""061012153439""","[6, 10, … 39]",6,10,12,15,34,39,"""06""","""10""","""12""","""15""","""34""","""39""","""06""","""07""","""19""","""21""","""28""","""40""",false,0,"""04_02_03_19_05""",-57
"""11/03/2025""","""1780""","""15 12 34 10 06 39""","""10 27 16 07 02 40""","""""","""""","""061012153439""","[6, 10, … 39]",6,10,12,15,34,39,"""06""","""10""","""12""","""15""","""34""","""39""","""06""","""10""","""12""","""15""","""34""","""39""",false,0,"""04_02_03_19_05""",-57
"""31/12/2024""","""1750""","""28 25 09 30 12 04""","""23 12 25 39 24 32""","""""","""""","""040912252830""","[4, 9, … 30]",4,9,12,25,28,30,"""04""","""09""","""12""","""25""","""28""","""30""","""01""","""10""","""16""","""24""","""34""","""35""",false,0,"""05_03_13_03_02""",61
"""31/12/2024""","""1750""","""28 25 09 30 12 04""","""23 12 25 39 24 32""","""""","""""","""040912252830""","[4, 9, … 30]",4,9,12,25,28,30,"""04""","""09""","""12""","""25""","""28""","""30""","""04""","""09""","""12""","""25""","""28""","""30""",false,0,"""05_03_13_03_02""",61
"""27/04/2024""","""1644""","""12 17 09 40 15 10""","""38 32 01 28 21 39""","""""","""""","""091012151740""","[9, 10, … 40]",9,10,12,15,17,40,"""09""","""10""","""12""","""15""","""17""","""40""","""10""","""16""","""22""","""27""","""28""","""37""",true,16,"""01_02_03_02_23""",26
"""27/04/2024""","""1644""","""12 17 09 40 15 10""","""38 32 01 28 21 39""","""""","""""","""091012151740""","[9, 10, … 40]",9,10,12,15,17,40,"""09""","""10""","""12""","""15""","""17""","""40""","""09""","""10""","""12""","""15""","""17""","""40""",true,16,"""01_02_03_02_23""",26
"""17/02/2024""","""1614""","""21 23 37 04 10 12""","""08 04 25 13 31 02""","""""","""""","""041012212337""","[4, 10, … 37]",4,10,12,21,23,37,"""04""","""10""","""12""","""21""","""23""","""37""","""06""","""13""","""18""","""21""","""35""","""37""",false,0,"""06_02_09_02_14""",58
"""17/02/2024""","""1614""","""21 23 37 04 10 12""","""08 04 25 13 31 02""","""""","""""","""041012212337""","[4, 10, … 37]",4,10,12,21,23,37,"""04""","""10""","""12""","""21""","""23""","""37""","""04""","""10""","""12""","""21""","""23""","""37""",false,0,"""06_02_09_02_14""",58


df_jumps_map FOR str_b3 = 12


jumps_map,count
str,u32
"""04_02_03_19_05""",2
"""06_02_09_02_14""",2
"""03_06_12_01_04""",2
"""08_01_03_18_04""",2
"""10_01_06_03_17""",2
…,…
"""06_05_09_09_10""",1
"""04_05_03_08_05""",1
"""06_01_12_05_11""",1


df_unijump FOR str_b3 = 12


unijump,count
i64,u32
26,8
-44,7
-15,6
19,6
51,5
…,…
67,1
107,1
-14,1


In [19]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='15'
df_viz FOR str_b4 = 15


post_str_b4,count,percent
str,u32,f64
"""15""",53,32.716049
"""22""",12,7.407407
"""25""",10,6.17284
"""30""",9,5.555556
"""21""",9,5.555556
…,…,…
"""09""",1,0.617284
"""10""",1,0.617284
"""11""",1,0.617284


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 15
Last 3 results last_results=['15', '21']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""04_06_03_22_01""",-89
"""11/03/2025""","""1780""","""15 12 34 10 06 39""","""10 27 16 07 02 40""","""""","""""","""061012153439""","[6, 10, … 39]",6,10,12,15,34,39,"""06""","""10""","""12""","""15""","""34""","""39""","""06""","""07""","""19""","""21""","""28""","""40""",false,0,"""04_02_03_19_05""",-57
"""11/03/2025""","""1780""","""15 12 34 10 06 39""","""10 27 16 07 02 40""","""""","""""","""061012153439""","[6, 10, … 39]",6,10,12,15,34,39,"""06""","""10""","""12""","""15""","""34""","""39""","""06""","""10""","""12""","""15""","""34""","""39""",false,0,"""04_02_03_19_05""",-57
"""07/11/2024""","""1727""","""15 36 22 09 04 13""","""32 06 23 01 11 39""","""""","""""","""040913152236""","[4, 9, … 36]",4,9,13,15,22,36,"""04""","""09""","""13""","""15""","""22""","""36""","""07""","""10""","""20""","""24""","""26""","""35""",false,0,"""05_04_02_07_14""",-13
"""07/11/2024""","""1727""","""15 36 22 09 04 13""","""32 06 23 01 11 39""","""""","""""","""040913152236""","[4, 9, … 36]",4,9,13,15,22,36,"""04""","""09""","""13""","""15""","""22""","""36""","""04""","""09""","""13""","""15""","""22""","""36""",false,0,"""05_04_02_07_14""",-13
"""05/10/2024""","""1713""","""14 07 15 25 40 04""","""17 33 23 26 30 04""","""""","""""","""040714152540""","[4, 7, … 40]",4,7,14,15,25,40,"""04""","""07""","""14""","""15""","""25""","""40""","""09""","""14""","""20""","""23""","""28""","""34""",true,16,"""03_07_01_10_15""",-44
"""05/10/2024""","""1713""","""14 07 15 25 40 04""","""17 33 23 26 30 04""","""""","""""","""040714152540""","[4, 7, … 40]",4,7,14,15,25,40,"""04""","""07""","""14""","""15""","""25""","""40""","""04""","""07""","""14""","""15""","""25""","""40""",true,16,"""03_07_01_10_15""",-44
"""06/07/2024""","""1674""","""15 09 07 23 24 10""","""05 09 39 10 01 21""","""""","""""","""070910152324""","[7, 9, … 24]",7,9,10,15,23,24,"""07""","""09""","""10""","""15""","""23""","""24""","""14""","""17""","""26""","""30""","""37""","""39""",false,0,"""02_01_05_08_01""",-3
"""06/07/2024""","""1674""","""15 09 07 23 24 10""","""05 09 39 10 01 21""","""""","""""","""070910152324""","[7, 9, … 24]",7,9,10,15,23,24,"""07""","""09""","""10""","""15""","""23""","""24""","""07""","""09""","""10""","""15""","""23""","""24""",false,0,"""02_01_05_08_01""",-3


df_jumps_map FOR str_b4 = 15


jumps_map,count
str,u32
"""03_03_04_02_01""",2
"""05_04_02_07_14""",2
"""07_03_03_01_17""",2
"""01_01_05_01_09""",2
"""02_09_02_03_13""",2
…,…
"""02_05_03_04_21""",1
"""01_04_01_22_02""",1
"""02_03_05_11_11""",1


df_unijump FOR str_b4 = 15


unijump,count
i64,u32
-44,10
-13,6
-24,6
-2,6
18,5
…,…
9,1
-89,1
-15,1


In [21]:
possible_b5, exclude_jump_maps_b5, last_post_b5 = plot_post_col(col='str_b5', post_col='post_str_b5')

Setting test_value='37'
df_viz FOR str_b5 = 37


post_str_b5,count,percent
str,u32,f64
"""37""",78,32.098765
"""30""",14,5.761317
"""33""",12,4.938272
"""27""",12,4.938272
"""36""",10,4.115226
…,…,…
"""21""",3,1.234568
"""22""",2,0.823045
"""16""",2,0.823045


:Overlay
   .Bars.I    :Bars   [post_str_b5]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 37
Last 3 results last_results=['25', '37']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""04_06_03_22_01""",-89
"""04/02/2025""","""1765""","""38 15 20 24 35 37""","""17 07 30 10 20 38""","""""","""""","""152024353738""","[15, 20, … 38]",15,20,24,35,37,38,"""15""","""20""","""24""","""35""","""37""","""38""","""03""","""08""","""13""","""17""","""25""","""26""",false,0,"""05_04_11_02_01""",48
"""04/02/2025""","""1765""","""38 15 20 24 35 37""","""17 07 30 10 20 38""","""""","""""","""152024353738""","[15, 20, … 38]",15,20,24,35,37,38,"""15""","""20""","""24""","""35""","""37""","""38""","""15""","""20""","""24""","""35""","""37""","""38""",false,0,"""05_04_11_02_01""",48
"""16/01/2025""","""1757""","""37 07 39 10 18 19""","""35 27 26 17 19 37""","""""","""""","""071018193739""","[7, 10, … 39]",7,10,18,19,37,39,"""07""","""10""","""18""","""19""","""37""","""39""","""02""","""03""","""16""","""18""","""26""","""28""",true,16,"""03_08_01_18_02""",-93
"""16/01/2025""","""1757""","""37 07 39 10 18 19""","""35 27 26 17 19 37""","""""","""""","""071018193739""","[7, 10, … 39]",7,10,18,19,37,39,"""07""","""10""","""18""","""19""","""37""","""39""","""07""","""10""","""18""","""19""","""37""","""39""",true,16,"""03_08_01_18_02""",-93
"""04/01/2025""","""1752""","""27 37 08 39 19 12""","""40 02 30 07 09 03""","""""","""""","""081219273739""","[8, 12, … 39]",8,12,19,27,37,39,"""08""","""12""","""19""","""27""","""37""","""39""","""04""","""06""","""07""","""08""","""27""","""38""",false,0,"""04_07_08_10_02""",-14
"""04/01/2025""","""1752""","""27 37 08 39 19 12""","""40 02 30 07 09 03""","""""","""""","""081219273739""","[8, 12, … 39]",8,12,19,27,37,39,"""08""","""12""","""19""","""27""","""37""","""39""","""08""","""12""","""19""","""27""","""37""","""39""",false,0,"""04_07_08_10_02""",-14
"""30/11/2024""","""1737""","""10 09 05 37 03 39""","""14 10 15 02 11 34""","""""","""""","""030509103739""","[3, 5, … 39]",3,5,9,10,37,39,"""03""","""05""","""09""","""10""","""37""","""39""","""09""","""10""","""14""","""28""","""35""","""36""",true,16,"""02_04_01_27_02""",-114
"""30/11/2024""","""1737""","""10 09 05 37 03 39""","""14 10 15 02 11 34""","""""","""""","""030509103739""","[3, 5, … 39]",3,5,9,10,37,39,"""03""","""05""","""09""","""10""","""37""","""39""","""03""","""05""","""09""","""10""","""37""","""39""",true,16,"""02_04_01_27_02""",-114


df_jumps_map FOR str_b5 = 37


jumps_map,count
str,u32
"""03_04_03_01_03""",2
"""18_14_01_02_02""",2
"""06_14_04_05_03""",2
"""17_12_03_03_03""",2
"""08_10_10_04_02""",2
…,…
"""05_24_02_02_03""",1
"""07_07_10_01_02""",1
"""07_22_06_01_03""",1


df_unijump FOR str_b5 = 37


unijump,count
i64,u32
-27,6
-22,5
48,5
-6,4
-85,4
…,…
-48,1
-89,1
-55,1


In [22]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b6', post_col='post_str_blast')

Setting test_value='38'
df_viz FOR str_b6 = 38


post_str_blast,count,percent
str,u32,f64
"""38""",209,36.795775
"""40""",62,10.915493
"""39""",53,9.330986
"""37""",45,7.922535
"""35""",30,5.28169
…,…,…
"""24""",2,0.352113
"""23""",2,0.352113
"""21""",2,0.352113


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b6 = 38
Last 3 results last_results=['26', '38']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""04_06_03_22_01""",-89
"""04/02/2025""","""1765""","""38 15 20 24 35 37""","""17 07 30 10 20 38""","""""","""""","""152024353738""","[15, 20, … 38]",15,20,24,35,37,38,"""15""","""20""","""24""","""35""","""37""","""38""","""03""","""08""","""13""","""17""","""25""","""26""",false,0,"""05_04_11_02_01""",48
"""04/02/2025""","""1765""","""38 15 20 24 35 37""","""17 07 30 10 20 38""","""""","""""","""152024353738""","[15, 20, … 38]",15,20,24,35,37,38,"""15""","""20""","""24""","""35""","""37""","""38""","""15""","""20""","""24""","""35""","""37""","""38""",false,0,"""05_04_11_02_01""",48
"""30/01/2025""","""1763""","""38 36 33 31 04 20""","""17 09 03 12 18 10""","""""","""""","""042031333638""","[4, 20, … 38]",4,20,31,33,36,38,"""04""","""20""","""31""","""33""","""36""","""38""","""04""","""14""","""26""","""27""","""28""","""33""",false,0,"""16_11_02_03_02""",-26
"""30/01/2025""","""1763""","""38 36 33 31 04 20""","""17 09 03 12 18 10""","""""","""""","""042031333638""","[4, 20, … 38]",4,20,31,33,36,38,"""04""","""20""","""31""","""33""","""36""","""38""","""04""","""20""","""31""","""33""","""36""","""38""",false,0,"""16_11_02_03_02""",-26
"""21/01/2025""","""1759""","""16 38 06 08 19 20""","""35 30 28 23 05 24""","""""","""""","""060816192038""","[6, 8, … 38]",6,8,16,19,20,38,"""06""","""08""","""16""","""19""","""20""","""38""","""10""","""14""","""19""","""25""","""27""","""34""",false,0,"""02_08_03_01_18""",2
"""21/01/2025""","""1759""","""16 38 06 08 19 20""","""35 30 28 23 05 24""","""""","""""","""060816192038""","[6, 8, … 38]",6,8,16,19,20,38,"""06""","""08""","""16""","""19""","""20""","""38""","""06""","""08""","""16""","""19""","""20""","""38""",false,0,"""02_08_03_01_18""",2
"""07/01/2025""","""1753""","""38 27 07 04 08 06""","""36 33 40 01 09 02""","""""","""""","""040607082738""","[4, 6, … 38]",4,6,7,8,27,38,"""04""","""06""","""07""","""08""","""27""","""38""","""02""","""11""","""14""","""18""","""22""","""39""",true,24,"""02_01_01_19_11""",-61
"""07/01/2025""","""1753""","""38 27 07 04 08 06""","""36 33 40 01 09 02""","""""","""""","""040607082738""","[4, 6, … 38]",4,6,7,8,27,38,"""04""","""06""","""07""","""08""","""27""","""38""","""04""","""06""","""07""","""08""","""27""","""38""",true,24,"""02_01_01_19_11""",-61


df_jumps_map FOR str_b6 = 38


jumps_map,count
str,u32
"""21_02_09_01_03""",2
"""03_05_15_02_09""",2
"""09_04_09_01_06""",2
"""01_10_01_04_21""",2
"""08_04_01_09_09""",2
…,…
"""02_04_09_12_10""",1
"""09_02_04_07_08""",1
"""08_04_02_09_02""",1


df_unijump FOR str_b6 = 38


unijump,count
i64,u32
2,13
-3,12
3,11
-18,10
-28,9
…,…
39,1
-58,1
23,1


In [23]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [24]:
parquet_file = './parquet_kbl_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []

  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)


  with open('./gn_cahe_v3.json', 'w') as file: 
    json.dump(lst, file)

  
  # dfRows = pl.DataFrame(
  #   lst,
  #   schema=[
  #     'fecha', 'sorteo', 'bolillas regulares', 'bolillas chauchamba', 'cc1', 'cc2',
  #     'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
  #     'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
  #     'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
  #     'is_rare_combination', 'reason',
  #     "jumps_map", "unijump"
  #   ],
  #   schema_overrides={
  #     'combo': pl.Array(pl.UInt8, 6),
  #     'b1': pl.UInt8,
  #     'b2': pl.UInt8,
  #     'b3': pl.UInt8,
  #     'b4': pl.UInt8,
  #     'b5': pl.UInt8,
  #     'b6': pl.UInt8,
  #     'reason': pl.UInt8
  #   },
  #   orient='row'
  # )

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 6),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

parquet_file='./parquet_kbl_v3.parquet' does not exists. creating
TOTAL_LINES=3838380 loaded to parquet_file='./parquet_kbl_v3.parquet'


id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,bool,u8,str,i64
"""010203040506""","[1, 2, … 6]",1,2,3,4,5,6,"""01""","""02""","""03""","""04""","""05""","""06""",true,8,"""01_01_01_01_01""",0
"""010203040507""","[1, 2, … 7]",1,2,3,4,5,7,"""01""","""02""","""03""","""04""","""05""","""07""",true,9,"""01_01_01_01_02""",1
"""010203040508""","[1, 2, … 8]",1,2,3,4,5,8,"""01""","""02""","""03""","""04""","""05""","""08""",true,8,"""01_01_01_01_03""",2
"""010203040509""","[1, 2, … 9]",1,2,3,4,5,9,"""01""","""02""","""03""","""04""","""05""","""09""",true,8,"""01_01_01_01_04""",3
"""010203040510""","[1, 2, … 10]",1,2,3,4,5,10,"""01""","""02""","""03""","""04""","""05""","""10""",true,8,"""01_01_01_01_05""",4
"""010203040511""","[1, 2, … 11]",1,2,3,4,5,11,"""01""","""02""","""03""","""04""","""05""","""11""",true,9,"""01_01_01_01_06""",5
"""010203040512""","[1, 2, … 12]",1,2,3,4,5,12,"""01""","""02""","""03""","""04""","""05""","""12""",true,8,"""01_01_01_01_07""",6
"""010203040513""","[1, 2, … 13]",1,2,3,4,5,13,"""01""","""02""","""03""","""04""","""05""","""13""",true,9,"""01_01_01_01_08""",7
"""010203040514""","[1, 2, … 14]",1,2,3,4,5,14,"""01""","""02""","""03""","""04""","""05""","""14""",true,8,"""01_01_01_01_09""",8


In [26]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  pl.col("str_b3").is_in(possible_b3),
  pl.col("str_b4").is_in(possible_b4),
  pl.col("str_b5").is_in(possible_b5),
  pl.col("str_b6").is_in(possible_blast),
  
  ~pl.col("str_b1").is_in(last_post_b1),
  ~pl.col("str_b2").is_in(last_post_b2),
  ~pl.col("str_b3").is_in(last_post_b3),
  ~pl.col("str_b4").is_in(last_post_b4),
  ~pl.col("str_b5").is_in(last_post_b5),
  ~pl.col("str_b6").is_in(last_post_blast),

  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_b5 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,bool,u8,str,i64
"""010515222434""","[1, 5, … 34]",1,5,15,22,24,34,"""01""","""05""","""15""","""22""","""24""","""34""",false,0,"""04_10_07_02_10""",8
"""010515222435""","[1, 5, … 35]",1,5,15,22,24,35,"""01""","""05""","""15""","""22""","""24""","""35""",false,0,"""04_10_07_02_11""",9
"""010515222437""","[1, 5, … 37]",1,5,15,22,24,37,"""01""","""05""","""15""","""22""","""24""","""37""",false,0,"""04_10_07_02_13""",11
"""010515222439""","[1, 5, … 39]",1,5,15,22,24,39,"""01""","""05""","""15""","""22""","""24""","""39""",false,0,"""04_10_07_02_15""",13
"""010515222734""","[1, 5, … 34]",1,5,15,22,27,34,"""01""","""05""","""15""","""22""","""27""","""34""",false,0,"""04_10_07_05_07""",-7
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""051325303439""","[5, 13, … 39]",5,13,25,30,34,39,"""05""","""13""","""25""","""30""","""34""","""39""",false,0,"""08_12_05_04_05""",-21
"""051325303440""","[5, 13, … 40]",5,13,25,30,34,40,"""05""","""13""","""25""","""30""","""34""","""40""",false,0,"""08_12_05_04_06""",-20
"""051325303637""","[5, 13, … 37]",5,13,25,30,36,37,"""05""","""13""","""25""","""30""","""36""","""37""",false,0,"""08_12_05_06_01""",-33


In [27]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,bool,u8,str,i64
"""011317243337""","[1, 13, … 37]",1,13,17,24,33,37,"""01""","""13""","""17""","""24""","""33""","""37""",false,0,"""12_04_07_09_04""",6
